In [1]:
import torch

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.utils import shuffle
import math
import sklearn

from sklearn.model_selection import train_test_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))



Using cuda device


In [3]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


In [6]:
signalz = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
lst = []
i = 2
while len(lst) < 50:
    i_round = int(i)
    if i_round not in lst:
        lst.append(i_round)
    i = i * 1.1
dim_lst = lst
dim_lst

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 16,
 17,
 19,
 21,
 23,
 26,
 28,
 31,
 34,
 38,
 42,
 46,
 51,
 56,
 61,
 68,
 74,
 82,
 90,
 99,
 109,
 120,
 132,
 145,
 160,
 176,
 194,
 213,
 234,
 258,
 284,
 312,
 343,
 378,
 415,
 457,
 503]

In [7]:
all_results = {}
all_point_of_fail = {}

In [8]:
def get_random_datapoint_t8(sig,prob, size = 2000, ):
    pts = np.random.normal(0.5, 0.05, size)
    if sig and (np.random.random() < prob):
        pts[0] = np.random.normal(0.6, 0.05, 1)
    return pts



In [23]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [27]:
def prep_data_t8(sig, bg):
    
    this_y_bg = np.zeros(len(bg))
    
    this_y_sig = np.ones(len(sig))
    
    """
    Shuffle + Train-Val-Test Split 
    """
    # Combine both data sets
    this_X = np.concatenate([bg, sig])
    this_y = np.concatenate([this_y_bg, this_y_sig])
    
    # Shuffle before we split
    this_X, this_y = shuffle(this_X, this_y)
    
    
    (X_train, X_val, 
     y_train, y_val) = train_test_split(this_X, this_y, test_size=0.2)
        
    X_val = np.ones(X_val.shape) * 0.5
    X_val[:,0] = 0.5 + 0.1*y_val
    
    # change Y to categorical Matrix
    Y_train = to_categorical(y_train.astype(int), num_classes=2)
    Y_val = to_categorical(y_val.astype(int), num_classes=2)
    
#     print('Training set size, distribution:')
#     print(X_train.shape)
#     print(np.unique(y_train,return_counts = True))
#     print('Validations set size, distribution:')
#     print(X_val.shape)
#     print(np.unique(y_val,return_counts = True))
    
    return X_train, X_val, Y_train,Y_val



In [44]:
class simple_dataset(Dataset):

    def __init__(self,X, y):
        
        assert len(X) == len(y)
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y)
                
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'X': self.X[idx], 'y': self.y[idx]}          
        
        return sample


In [74]:
#[100, 50]

class MyNet_M(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_M, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100)
        self.fc2 = torch.nn.Linear(100, 50)
        self.fc3 = torch.nn.Linear(50, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

#model.apply(weight_init)


class MyNet_S(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_S, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 50)
        self.fc2 = torch.nn.Linear(50, 25)
        self.fc3 = torch.nn.Linear(25, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
    
class MyNet_L(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_L, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100)
        self.fc2 = torch.nn.Linear(100, 100)
        self.fc3 = torch.nn.Linear(100, 50)
        self.fc4 = torch.nn.Linear(50, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x
    
    
class MyNet_XL(torch.nn.Module):
    def __init__(self, input_size):
        
        super(MyNet_XL, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 200)
        self.fc2 = torch.nn.Linear(200, 200)
        self.fc3 = torch.nn.Linear(200, 100)
        self.fc4 = torch.nn.Linear(100, 2)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [17]:
hidden_sizes_t8 = [100, 50]
batch_size_t8 = 100
MAX_EPOCHS = 20
loss_fn = torch.nn.CrossEntropyLoss()


In [67]:
def train_epoch(model, dataloader):
    
    model.train()
    
    for batch in dataloader:

        x, y = batch['X'], batch['y']
        x, y = x.to(device), y.to(device)

        # do the forward/backward pass
        model.zero_grad()
        output = model(x)
        loss = loss_fn(output,y)
        loss.backward()
        optimizer.step()

        del x
        del y
    
def validate_acc(model, Xval, Yval):
    
    X = torch.tensor(Xval).float().to(device)
    y_val = torch.tensor(Yval).float().to(device)
    Y_pred = model(X)
    
    Y_pred_rnd = (Y_pred >= 0.5)
    
    matches = sum(Y_pred_rnd == y_val)
    
    acc = matches[0]/(len(Y_pred))

    return acc
    



In [69]:
import pickle

In [ ]:
all_point_of_fail = {}

for sig_ratio in signalz:
    mock_signal_t8 = np.array([get_random_datapoint_t8(True, sig_ratio) for _ in range(20000)])
    mock_bg_t8 = np.array([get_random_datapoint_t8(False, sig_ratio) for _ in range(20000)])
    
    
    X_train_t8, X_val_t8, Y_train_t8,Y_val_t8 = prep_data_t8(mock_signal_t8, mock_bg_t8)
    
    
    print(f"sig ratio is {sig_ratio}")
    
    for dimension in dim_lst:
        print(f"dim is {dimension}")
        
        this_dim = {
            'sig_ratio': sig_ratio,
            'dimension': dimension,
        }
        
        this_results = []
        
        passed = False
        
        Xtrain, Xval = X_train_t8[:,:dimension], X_val_t8[:,:dimension]
        Ytrain, Yval = Y_train_t8,Y_val_t8
        
        train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        model = MyNet(dimension)
        model.to(device)
        
        for trial in tqdm(range(7)):
            
            train_loader = DataLoader(train_dataset, batch_size=20)
            
            model.apply(weights_init)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):
                
                train_epoch(model, train_loader)
                
                val_acc = validate_acc(model, Xval, Yval)
                                
                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True
                
            else:
                this_results.append([dimension,-1])
                
        print(this_results)
        this_dim['result'] = this_results
        
        with open(f'data/sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
            pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        if passed == False:
            all_point_of_fail[sig_ratio] = dimension
            break
            
        del model
            


sig ratio is 0.001
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:54<00:00, 33.55s/it]


[[2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.005
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:53<00:00, 33.41s/it]


[[2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.01
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:43<00:00,  6.24s/it]


[[2, 19], [2, 0], [2, 0], [2, 0], [2, 0], [2, 0], [2, 0]]
dim is 3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:23<00:00,  3.34s/it]


[[3, 5], [3, 0], [3, 1], [3, 0], [3, 0], [3, 0], [3, 1]]
dim is 4


  0%|                                                                                                                                                                                                                                                               | 0/7 [00:00<?, ?it/s]

In [75]:
for sig_ratio in signalz:
    mock_signal_t8 = np.array([get_random_datapoint_t8(True, sig_ratio) for _ in range(20000)])
    mock_bg_t8 = np.array([get_random_datapoint_t8(False, sig_ratio) for _ in range(20000)])
    
    
    X_train_t8, X_val_t8, Y_train_t8,Y_val_t8 = prep_data_t8(mock_signal_t8, mock_bg_t8)
    
    
    print(f"sig ratio is {sig_ratio}")
    
    for dimension in dim_lst:
        print(f"dim is {dimension}")
        
        this_dim = {
            'sig_ratio': sig_ratio,
            'dimension': dimension,
        }
        
        this_results = []
        
        passed = False
        
        Xtrain, Xval = X_train_t8[:,:dimension], X_val_t8[:,:dimension]
        Ytrain, Yval = Y_train_t8,Y_val_t8
        
        train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        model = MyNet_S(dimension)
        model.to(device)
        
        for trial in tqdm(range(4)):
            
            train_loader = DataLoader(train_dataset, batch_size=20)
            
            model.apply(weights_init)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):
                
                train_epoch(model, train_loader)
                
                val_acc = validate_acc(model, Xval, Yval)
                                
                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True
                
            else:
                this_results.append([dimension,-1])
                
        print(this_results)
        this_dim['result'] = this_results
        
        with open(f'data2/model_S_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
            pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        if passed == False:
            break
            
        del model

sig ratio is 0.001
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:16<00:00, 34.21s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.005
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:14<00:00, 33.51s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.01
dim is 2


...

[[34, 3], [34, 0], [34, 0], [34, 0]]
dim is 38


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.96s/it]


[[38, 3], [38, 0], [38, 0], [38, 0]]
dim is 42


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:14<00:00, 33.52s/it]

[[42, -1], [42, -1], [42, -1], [42, -1]]


In [76]:
for sig_ratio in signalz:
    mock_signal_t8 = np.array([get_random_datapoint_t8(True, sig_ratio) for _ in range(20000)])
    mock_bg_t8 = np.array([get_random_datapoint_t8(False, sig_ratio) for _ in range(20000)])
    
    
    X_train_t8, X_val_t8, Y_train_t8,Y_val_t8 = prep_data_t8(mock_signal_t8, mock_bg_t8)
    
    
    print(f"sig ratio is {sig_ratio}")
    
    for dimension in dim_lst:
        print(f"dim is {dimension}")
        
        this_dim = {
            'sig_ratio': sig_ratio,
            'dimension': dimension,
        }
        
        this_results = []
        
        passed = False
        
        Xtrain, Xval = X_train_t8[:,:dimension], X_val_t8[:,:dimension]
        Ytrain, Yval = Y_train_t8,Y_val_t8
        
        train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        model = MyNet_M(dimension)
        model.to(device)
        
        for trial in tqdm(range(4)):
            
            train_loader = DataLoader(train_dataset, batch_size=20)
            
            model.apply(weights_init)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):
                
                train_epoch(model, train_loader)
                
                val_acc = validate_acc(model, Xval, Yval)
                                
                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True
                
            else:
                this_results.append([dimension,-1])
                
        print(this_results)
        this_dim['result'] = this_results
        
        with open(f'data2/model_M_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
            pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        if passed == False:
            break
            
        del model

sig ratio is 0.001
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:14<00:00, 33.63s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.005
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:13<00:00, 33.41s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.01
dim is 2


...

[[46, 9], [46, 0], [46, 0], [46, 0]]
dim is 51


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.10s/it]


[[51, 8], [51, 0], [51, 0], [51, 0]]
dim is 56


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:15<00:00, 34.00s/it]

[[56, -1], [56, -1], [56, -1], [56, -1]]


In [77]:
for sig_ratio in signalz:
    mock_signal_t8 = np.array([get_random_datapoint_t8(True, sig_ratio) for _ in range(20000)])
    mock_bg_t8 = np.array([get_random_datapoint_t8(False, sig_ratio) for _ in range(20000)])
    
    
    X_train_t8, X_val_t8, Y_train_t8,Y_val_t8 = prep_data_t8(mock_signal_t8, mock_bg_t8)
    
    
    print(f"sig ratio is {sig_ratio}")
    
    for dimension in dim_lst:
        print(f"dim is {dimension}")
        
        this_dim = {
            'sig_ratio': sig_ratio,
            'dimension': dimension,
        }
        
        this_results = []
        
        passed = False
        
        Xtrain, Xval = X_train_t8[:,:dimension], X_val_t8[:,:dimension]
        Ytrain, Yval = Y_train_t8,Y_val_t8
        
        train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        model = MyNet_L(dimension)
        model.to(device)
        
        for trial in tqdm(range(4)):
            
            train_loader = DataLoader(train_dataset, batch_size=20)
            
            model.apply(weights_init)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):
                
                train_epoch(model, train_loader)
                
                val_acc = validate_acc(model, Xval, Yval)
                                
                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True
                
            else:
                this_results.append([dimension,-1])
                
        print(this_results)
        this_dim['result'] = this_results
        
        with open(f'data2/model_L_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
            pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        if passed == False:
            break
            
        del model

sig ratio is 0.001
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:39<00:00, 39.80s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.005
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:40<00:00, 40.09s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.01
dim is 2


...

[[34, 4], [34, 0], [34, 0], [34, 0]]
dim is 38


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:18<00:00,  4.53s/it]


[[38, 5], [38, 0], [38, 0], [38, 0]]
dim is 42


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:41<00:00, 40.31s/it]

[[42, -1], [42, -1], [42, -1], [42, -1]]


In [78]:
for sig_ratio in signalz:
    mock_signal_t8 = np.array([get_random_datapoint_t8(True, sig_ratio) for _ in range(20000)])
    mock_bg_t8 = np.array([get_random_datapoint_t8(False, sig_ratio) for _ in range(20000)])
    
    
    X_train_t8, X_val_t8, Y_train_t8,Y_val_t8 = prep_data_t8(mock_signal_t8, mock_bg_t8)
    
    
    print(f"sig ratio is {sig_ratio}")
    
    for dimension in dim_lst:
        print(f"dim is {dimension}")
        
        this_dim = {
            'sig_ratio': sig_ratio,
            'dimension': dimension,
        }
        
        this_results = []
        
        passed = False
        
        Xtrain, Xval = X_train_t8[:,:dimension], X_val_t8[:,:dimension]
        Ytrain, Yval = Y_train_t8,Y_val_t8
        
        train_dataset = simple_dataset(Xtrain, Ytrain)
        
        
        model = MyNet_XL(dimension)
        model.to(device)
        
        for trial in tqdm(range(4)):
            
            train_loader = DataLoader(train_dataset, batch_size=20)
            
            model.apply(weights_init)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            train_acc = 0
            val_acc = 0
            success = 0
            for epoch in (range(MAX_EPOCHS)):
                
                train_epoch(model, train_loader)
                
                val_acc = validate_acc(model, Xval, Yval)
                                
                if (val_acc >= 0.6):
                    success = 1
                    this_results.append([dimension,epoch])
                    break
            if success:
                passed = True
                
            else:
                this_results.append([dimension,-1])
                
        print(this_results)
        this_dim['result'] = this_results
        
        with open(f'data2/model_XL_sig_{sig_ratio}_dim_{dimension}.pickle', 'wb') as handle:
            pickle.dump(this_dim, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        if passed == False:
            break
            
        del model

sig ratio is 0.001
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:40<00:00, 40.02s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.005
dim is 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:40<00:00, 40.02s/it]


[[2, -1], [2, -1], [2, -1], [2, -1]]
sig ratio is 0.01
dim is 2


...

[[26, 2], [26, 0], [26, 0], [26, 0]]
dim is 28


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.03s/it]


[[28, 4], [28, 0], [28, 0], [28, 0]]
dim is 31


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:40<00:00, 40.07s/it]

[[31, -1], [31, -1], [31, -1], [31, -1]]
